In [1]:
import pandas as pd
import io
from tqdm import tqdm
from time import sleep
import numpy as np

In [2]:
filepath = '../data/NAD_r11.txt' # too large to push onto Git, but is part of the download from here: https://nationaladdressdata.s3.amazonaws.com/NAD_r12_TXT.zip

In [3]:
first_line = None

with open(filepath) as f:
    first_line = f.readline()

print(first_line)

OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,Bulk_Plus4,StN_PreMod,StN_PreDir,StN_PreTyp,StN_PreSep,StreetName,StN_PosTyp,StN_PosDir,StN_PosMod,AddNum_Pre,Add_Number,AddNum_Suf,LandmkPart,LandmkName,Building,Floor,Unit,Room,Addtl_Loc,Milepost,Longitude,Latitude,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired



In [4]:
total_lines = sum(1 for line in open(filepath))
print('Total number of lines: %s' % total_lines)

Total number of lines: 67357690


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
step = 1000
dfs = []

with tqdm(total=total_lines-1) as pbar:
    with open(filepath) as f:
        f.readline() # skip first line
        i = 0
        lines = ''
        for line in f:
            pbar.update(1)
            i += 1
            lines += line
            if i > 1000:

                df = pd.read_csv(io.StringIO(first_line + lines), low_memory=False)
                pdf = df[['State', 'County', 'Zip_Code', 'Add_Number', 'StreetName', 'Longitude', 'Latitude']]
                pdf = pdf.astype(str)                
                pdf['Address'] = pdf['Add_Number'] + ' ' + pdf['StreetName'] + ' ' + pdf['State'] + ' ' + pdf['Zip_Code']
                dfs.append(pdf)
                i = 0
                lines = ''
                

100%|███████████████████████████| 67357689/67357689 [07:08<00:00, 157052.86it/s]


In [7]:
# Column descriptors: https://www.transportation.gov/sites/dot.gov/files/docs/mission/gis/national-address-database/308816/nad-schema-v1.pdf

In [8]:
len(dfs)

67290

In [9]:
df = pd.concat(dfs)

In [10]:
df

,State,County,Zip_Code,Add_Number,StreetName,Longitude,Latitude,Address
0,AZ,Coconino,86336,95,LYNX,-111.771770637,34.8280627940001,95 LYNX AZ 86336
1,AZ,Coconino,86336,33,EAGLE,-111.769395183,34.828455976,33 EAGLE AZ 86336
2,AZ,Coconino,86336,141,VISTA BONITA,-111.777281707,34.828623958,141 VISTA BONITA AZ 86336
3,AZ,Coconino,86336,109,BADGER,-111.775765885,34.8291778600001,109 BADGER AZ 86336
4,AZ,Coconino,86336,30,FAWN,-111.773383133,34.824991696,30 FAWN AZ 86336
...,...,...,...,...,...,...,...,...
996,PA,Westmoreland,nan,3073,Audrey,-79.57448066349987,40.27125643079627,3073 Audrey PA nan
997,PA,Westmoreland,nan,3075,Audrey,-79.5743142038811,40.27121039055359,3075 Audrey PA nan
998,PA,Westmoreland,nan,3079,Audrey,-79.57414191689105,40.27118391184811,3079 Audrey PA nan
999,PA,Westmoreland,nan,3081,Audrey,-79.57395784490284,40.27119169192235,3081 Audrey PA nan


In [11]:
len(df[df['Zip_Code'] == 'nan']['State'].unique())

33

In [19]:
tdf = df[df['Zip_Code'] != 'nan']
tdf

,State,County,Zip_Code,Add_Number,StreetName,Longitude,Latitude,Address
0,AZ,Coconino,86336,95,LYNX,-111.771770637,34.8280627940001,95 LYNX AZ 86336
1,AZ,Coconino,86336,33,EAGLE,-111.769395183,34.828455976,33 EAGLE AZ 86336
2,AZ,Coconino,86336,141,VISTA BONITA,-111.777281707,34.828623958,141 VISTA BONITA AZ 86336
3,AZ,Coconino,86336,109,BADGER,-111.775765885,34.8291778600001,109 BADGER AZ 86336
4,AZ,Coconino,86336,30,FAWN,-111.773383133,34.824991696,30 FAWN AZ 86336
...,...,...,...,...,...,...,...,...
104,PA,Westmoreland,15601.0,605,Farview,-79.52236881714968,40.290077006924825,605 Farview PA 15601.0
105,PA,Westmoreland,15601.0,607,Farview,-79.52241451804144,40.29008212001564,607 Farview PA 15601.0
106,PA,Westmoreland,15601.0,597,Farview,-79.52211734366738,40.290106697381,597 Farview PA 15601.0
553,PA,Westmoreland,15601.0,598,Farview,-79.52401315183624,40.29069422586329,598 Farview PA 15601.0


In [26]:
tdf['zip_num'] = tdf['Zip_Code'].apply(lambda x: np.isreal(x)) #

In [29]:
l = []

for zip in tqdm(list(tdf['Zip_Code'])):
    try:
        num = str(int(float(zip)))
    except:
        print(zip)
        num = None
    l.append(num)

  5%|█▍                         | 3456764/62897908 [00:01<00:41, 1416385.87it/s]

IA


  7%|█▊                         | 4361685/62897908 [00:02<00:28, 2024327.34it/s]

5-156


 18%|████▌                     | 11111315/62897908 [00:04<00:18, 2844637.95it/s]

...

385O6
TN
373O3
TN
TN
TN
CLAIB
VILLA
NEW T


 99%|█████████████████████████▋| 62256396/62897908 [00:22<00:00, 2852570.04it/s]

Acme
Acme
Acme
Acme


100%|██████████████████████████| 62897908/62897908 [00:22<00:00, 2756049.59it/s]


OUT
OUT


In [30]:
tdf['Zip_Code'] = l

In [32]:
tdf = tdf[tdf['Zip_Code']!= None] # removing all incorrect zip codes from the NAD

In [33]:
tdf['Address'] = tdf['Add_Number'] + ' ' + tdf['StreetName'] + ' ' + tdf['State'] + ' ' + tdf['Zip_Code']

In [ ]:
len(df['State'].unique())

In [ ]:
len(tdf['State'].unique())

In [34]:
tdf.rename(columns={
    'State': 'state', 
    'County': 'county', 
    'Zip_Code': 'zip',
    'Add_Number': 'add_number',
    'StreetName': 'street_name',
    'Longitude': 'longitude',
    'Latitude': 'latitude',
    'Address': 'address',
}, inplace=True)

In [36]:
tdf

,state,county,zip,add_number,street_name,longitude,latitude,address,zip_num
0,AZ,Coconino,86336,95,LYNX,-111.771770637,34.8280627940001,95 LYNX AZ 86336,False
1,AZ,Coconino,86336,33,EAGLE,-111.769395183,34.828455976,33 EAGLE AZ 86336,False
2,AZ,Coconino,86336,141,VISTA BONITA,-111.777281707,34.828623958,141 VISTA BONITA AZ 86336,False
3,AZ,Coconino,86336,109,BADGER,-111.775765885,34.8291778600001,109 BADGER AZ 86336,False
4,AZ,Coconino,86336,30,FAWN,-111.773383133,34.824991696,30 FAWN AZ 86336,False
...,...,...,...,...,...,...,...,...,...
104,PA,Westmoreland,15601,605,Farview,-79.52236881714968,40.290077006924825,605 Farview PA 15601,False
105,PA,Westmoreland,15601,607,Farview,-79.52241451804144,40.29008212001564,607 Farview PA 15601,False
106,PA,Westmoreland,15601,597,Farview,-79.52211734366738,40.290106697381,597 Farview PA 15601,False
553,PA,Westmoreland,15601,598,Farview,-79.52401315183624,40.29069422586329,598 Farview PA 15601,False


In [2]:
# tdf = pd.read_csv('../data/nad.csv.xz')

In [3]:
del tdf['zip_num']

In [6]:
counties = list(tdf['county'].unique())

In [8]:
for county in tqdm(counties):
    pdf = tdf[tdf['county'] == county]
    state = list(pdf['state'].unique())[0]
    pdf.to_csv('../data/%s_%s.csv.xz' % (state, county), index=False)

100%|█████████████████████████████████████| 1082/1082 [3:03:55<00:00, 10.20s/it]
